In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pwd

/content


In [5]:
# Load the list from the binary file
import pickle

file_name = "data_storage.bin"

with open(file_name, "rb") as file:
    data_storage = pickle.load(file)

print("Data loaded from binary file:")
print(data_storage)

Data loaded from binary file:
[['no force', -348.0, -349.0, -343.0, -335.0, -327.0, -325.0, -334.0, -335.0, -345.0, -356.0, -343.0, -308.0, -293.0, -332.0, -389.0, -396.0, -339.0, -282.0, -284.0, -333.0, -372.0, -370.0, -347.0, -332.0, -328.0, -325.0, -322.0, -330.0, -345.0, -355.0, -350.0, -337.0, -330.0, -329.0, -330.0, -332.0, -335.0, -342.0, -347.0, -346.0, -340.0, -333.0, -328.0, -327.0, -332.0, -338.0, -346.0, -349.0, -345.0, -337.0, -328.0, -324.0, -328.0, -337.0, -345.0, -350.0, -348.0, -339.0, -329.0, -323.0, -326.0, -335.0, -344.0, -350.0, -349.0, -341.0, -332.0, -325.0, -325.0, -332.0, -341.0, -349.0, -349.0, -343.0, -335.0, -327.0, -325.0, -329.0, -337.0, -346.0, -350.0, -346.0, -337.0, -329.0, -324.0, -327.0, -335.0, -343.0, -349.0, -348.0, -341.0, -332.0, -325.0, -325.0, -332.0, -341.0, -348.0, -349.0, -344.0, -336.0, -327.0, -324.0, -329.0, -337.0, -346.0, -350.0, -347.0, -338.0, -329.0, -324.0, -326.0, -335.0, -343.0, -349.0, -348.0, -341.0, -332.0, -325.0, -325.0, -331

# Random Forest Prediction

In [6]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score


# Step 1: Preprocess the data
min_length = min(len(session[1:]) for session in data_storage)

features = []
labels = []

for session in data_storage:
    label = session[0]
    data = session[1:]

    # Truncate or pad to minimum length
    truncated_data = data[:min_length]

    labels.append(label)
    features.append(truncated_data)

X = np.array(features, dtype=np.float32)
y = np.array(labels)

# Step 2: Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Step 3: Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Step 4: Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Step 5: Train the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Step 6: Predict and evaluate
y_pred = rf_model.predict(X_test)

# Print accuracy and classification report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Decode and print predicted and true labels
predicted_labels = label_encoder.inverse_transform(y_pred)
true_labels = label_encoder.inverse_transform(y_test)

print("Predicted Labels:", predicted_labels)
print("True Labels:", true_labels)


Accuracy: 0.47058823529411764
Classification Report:
              precision    recall  f1-score   support

        hold       0.00      0.00      0.00         1
      medium       0.43      0.75      0.55         4
    no force       0.43      0.75      0.55         4
         put       0.00      0.00      0.00         1
       small       0.00      0.00      0.00         5
      strong       1.00      1.00      1.00         2

    accuracy                           0.47        17
   macro avg       0.31      0.42      0.35        17
weighted avg       0.32      0.47      0.37        17

Predicted Labels: ['medium' 'no force' 'small' 'no force' 'no force' 'medium' 'strong'
 'medium' 'strong' 'medium' 'medium' 'no force' 'medium' 'medium'
 'no force' 'no force' 'no force']
True Labels: ['no force' 'no force' 'medium' 'small' 'small' 'hold' 'strong' 'medium'
 'strong' 'small' 'put' 'no force' 'medium' 'medium' 'small' 'small'
 'no force']


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Now let us try autologon

In [1]:
! pip install autogluon

In [19]:
# 定义需要移除的标签
labels_to_remove = ['put', 'hold']

# 过滤掉指定标签的样本
filtered_data_storage = [sample for sample in data_storage if sample[0] not in labels_to_remove]

# 计算最短特征长度
min_length = min(len(sample) - 1 for sample in filtered_data_storage)

# 准备数据
data = []
for sample in filtered_data_storage:
    label = sample[0]
    features = sample[1:]
    # 截断特征到最短长度
    truncated_features = features[:min_length]
    data.append([label] + truncated_features)

# 创建 DataFrame
columns = ['label'] + [f'feature_{i+1}' for i in range(min_length)]
df = pd.DataFrame(data, columns=columns)


In [20]:
# 划分特征和标签
X = df.drop(columns=['label'])
y = df['label']

# 划分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 合并特征和标签以供 AutoGluon 使用
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)


In [21]:
from sklearn.model_selection import train_test_split

# Split features and labels
X = df.drop(columns=['label'])
y = df['label']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Combine features and labels for AutoGluon
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)


In [22]:
import pandas as pd
from autogluon.tabular import TabularPredictor

# 初始化预测器
predictor = TabularPredictor(label='label').fit(train_data)




No path specified. Models will be saved in: "AutogluonModels/ag-20241124_010301"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.00 GB / 12.67 GB (78.9%)
Disk Space Avail:   183.96 GB / 225.83 GB (81.5%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : 

In [23]:
# 评估模型性能
performance = predictor.evaluate(test_data)

# 显示评估指标
print("模型在测试集上的性能：")
for metric, score in performance.items():
    print(f"{metric}: {score:.4f}")


模型在测试集上的性能：
accuracy: 0.5333
balanced_accuracy: 0.6250
mcc: 0.5089
